## Pricing Asian Style Options
In this cript we show how to use classes in *QMCPy* for Monte Carlo option pricing of options with Asian style payoffs and European exercise.
 
- The payoff depends on the whole asset price path, not only on the terminal asset price.
- The option is only exercised at expiry, unlike American options, which can be exercised at any time before expiry.


In [2]:
# Import necessary packages
import qmcpy as qp
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import time

/Users/yding/opt/anaconda3/envs/qmcpy/lib/python3.9/site-packages/scipy/__init__.py:155


### European Options

In [27]:
initPrice = 120 # initial stock price
interest = 0.02 # risk-free interest rate
vol = 0.5 # volatility
callput = 'call' # call options
strike = 130 # strike price
tfinal = 1/4 # mature time
d = 12 # number of observations
absTol = 0.05 # absolute tolerance of a nickel
relTol = 0 # zero relative tolerance
sampleSize = 10**6 # number of smaple size
absTol = 0.05 # absolute tolerance of a nickel

In [28]:
EuroCall = qp.EuropeanOption(qp.IIDStdUniform(dimension=d,seed=7), volatility= vol,start_price= initPrice,
                       strike_price=strike, interest_rate = interest,t_final=1/4,call_put=callput)
print("The exact price of this European Call Option is ",f"{EuroCall.get_exact_value():.4f}")
sc = qp.CubMCCLT(EuroCall,absTol)
solution, data = sc.integrate()
print("To reach the absolute error tolerance = ",absTol, ", we need to generate ", f"{int(data.n_total):,}",
      "iid points, the estimation of the fair price is",f"{solution:.4f}")

The exact price of this European Call Option is  8.2779
To reach the absolute error tolerance =  0.05 , we need to generate  1,346,469 iid points,the estimation of the fair price is 8.2671


### Arithmetic Mean Options
The payoff of the arithmetic mean option depends on the average of the
stock price, not the final stock price.  Here are the discounted payoffs:

$$\begin{array}{rcc}
 & \textbf{call} & \textbf{put} \\ \hline
\textbf{payoff} & 
\displaystyle \max\biggl(\frac 1d \sum_{j=1}^d S(jT/d) - K,0 \biggr)\mathsf{e}^{-rT} & 
\displaystyle \max\biggl(K - \frac 1d \sum_{j=1}^d S(jT/d),0 \biggr)\mathsf{e}^{-rT} 
\end{array}
$$

In [29]:
meanType = 'arithmetic'
ArithMeanCall = qp.AsianOption(qp.IIDStdUniform(dimension=d,seed=7), volatility= vol,start_price= initPrice, 
                               strike_price=strike, interest_rate = interest,t_final=1/4,call_put=callput,mean_type=meanType)
sc = qp.CubMCCLT(ArithMeanCall,absTol)
solution, data = sc.integrate()
print("To reach the absolute error tolerance = ",absTol, ", we need to generate ", f"{int(data.n_total):,}",
      "iid points, the estimation of the fair price is",f"{solution:.4f}")

To reach the absolute error tolerance =  0.05 , we need to generate  287,434 iid points, the estimation of the fair price is 3.3908


The price of the Asian arithmetic mean call option is smaller than the price of the European call option.  

We may also price the Asian arithmetic mean put option as follows:

In [30]:
ArithMeanPut = ArithMeanCall
ArithMeanPut.call_put = 'put'
sc = qp.CubMCCLT(ArithMeanPut,absTol)
solution, data = sc.integrate()
print("To reach the absolute error tolerance = ",absTol, ", we need to generate ", f"{int(data.n_total):,}",
      "iid points, the estimation of the fair price is",f"{solution:.4f}")

To reach the absolute error tolerance =  0.05 , we need to generate  524,796 iid points, the estimation of the fair price is 13.0118


Note that the price is greater.  This is because one strike price is
above the initial price, making the expected payoff greater.

In the limit of continuous monitoring $d \to \infty$, the payoff is 

$$
\begin{array}{rcc}
& \textbf{call} & \textbf{put} \\ \hline
\textbf{payoff} & 
\displaystyle \max\biggl(\frac 1T \int_{0}^T S(t) \, {\rm d} t - K,0 \biggr)\mathsf{e}^{-rT} & 
\displaystyle \max\biggl(K - \frac 1T \int_{0}^T S(t) \, {\rm d} t,0 \biggr)\mathsf{e}^{-rT} 
\end{array} 
$$

Such an option can be approximated by taking smaller time steps:

In [31]:
d = 62 # Daily Monitoring
ArithMeanCall.sampler = qp.IIDStdUniform(dimension=d,seed=7)
ArithMeanCall.call_put = 'call'
ArithMeanCall = qp.AsianOption(qp.IIDStdUniform(dimension=d,seed=7), volatility= vol,start_price= initPrice, 
                               strike_price=strike, interest_rate = interest,t_final=1/4,call_put=callput,mean_type=meanType)
sc = qp.CubMCCLT(ArithMeanCall,absTol)
solution, data = sc.integrate()
print("To reach the absolute error tolerance = ",absTol, ", we need to generate ", f"{int(data.n_total):,}",
      "iid points, the estimation of the fair price is",f"{solution:.4f}")

To reach the absolute error tolerance =  0.05 , we need to generate  262,466 iid points, the estimation of the fair price is 3.3739


The price is a bit lower, and the time is longer because more time steps are needed, which means more random variables are needed.

### Geometric Mean Options
One can also base the payoff on a geometric mean rather than an arithmetic mean.  Such options have a closed form solution.  

The price of
a geometric mean 
$
\begin{Bmatrix} 
\text{call} \\ 
\text{put}
\end{Bmatrix}$
option is 
$\begin{Bmatrix} \le \\
\ge \end{Bmatrix}$
the
price of an arithmetic mean $\begin{Bmatrix} \text{call} \\\text{put} \end{Bmatrix}$ option because a geometric mean is smaller
than an arithmetic mean.

In [32]:
GeoMeanPut = ArithMeanCall
d = 12 #Weekly Monitoring for three months
GeoMeanPut.sampler = qp.IIDStdUniform(dimension=d,seed=7) 
GeoMeanPut.mean_type = 'geometric'
GeoMeanPut.call_put = 'put'
sc = qp.CubMCCLT(GeoMeanPut,absTol)
solution, data = sc.integrate()
print("To reach the absolute error tolerance = ",absTol, ", we need to generate ", f"{int(data.n_total):,}",
      "iid points, the estimation of the fair price is",f"{solution:.4f}")

To reach the absolute error tolerance =  0.05 , we need to generate  598,881 iid points, the estimation of the fair price is 13.4233


In [33]:
GeoMeanCall = GeoMeanPut
GeoMeanPut.call_put = 'call'
sc = qp.CubMCCLT(GeoMeanCall,absTol)
solution, data = sc.integrate()
print("To reach the absolute error tolerance = ",absTol, ", we need to generate ", f"{int(data.n_total):,}",
      "iid points, the estimation of the fair price is",f"{solution:.4f}")

To reach the absolute error tolerance =  0.05 , we need to generate  194,906 iid points, the estimation of the fair price is 3.1674


### Barrier Option
In barrier options the payoff only occurs if the asset price crosses or
fails to cross a barrier, $b$

$$
\begin{array}{rcc}
 & \textbf{up} (S(0) < b) & \textbf{down} (S(0) > b) \\ \hline
 \textbf{in} & \text{active if } S(t) \ge b & \text{active if } S(t) \le
 b \\
 \textbf{out} & \text{inactive if } S(t) \ge b & \text{inactive if } S(t) \le
 b 
 \end{array}
$$

For the barrier option with a European call type payoff, this corresponds to 

$$
 \begin{array}{rcc}
 \textbf{payoff} & \textbf{up} (S(0) < b) & \textbf{down} (S(0) > b) \\ \hline
 \textbf{in} & 
 1_{[b,\infty)}(\max_{0 \le t \le T} S(t)) \max(S(T)-K,0)\mathsf{e}^{-rT} & 
 1_{[0,b]}(\min_{0 \le t \le T} S(t)) \max(S(T)-K,0)\mathsf{e}^{-rT} \\
 \textbf{out} & 1_{[0,b)}(\max_{0 \le t \le T} S(t)) \max(S(T)-K,0)\mathsf{e}^{-rT} & 
 1_{[b,\infty)}(\min_{0 \le t \le T} S(t)) \max(S(T)-K,0)\mathsf{e}^{-rT}
 \end{array}
$$

In [35]:
barrier = 150
inOutType = 'in'
BarrierUpInCall = qp.BarrierOption(qp.IIDStdUniform(dimension=d,seed=7), volatility= vol,start_price= initPrice,
                       strike_price=strike, barrier_price=barrier,interest_rate = interest,t_final=1/4,
                       call_put='call', in_out = inOutType)
sc = qp.CubMCCLT(BarrierUpInCall,absTol)
solution, data = sc.integrate()
print("To reach the absolute error tolerance = ",absTol, ", we need to generate ", f"{int(data.n_total):,}",
      "iid points, the estimation of the fair price is",f"{solution:.4f}")

To reach the absolute error tolerance =  0.05 , we need to generate  1,360,734 iid points, the estimation of the fair price is 7.3984


Note that this price is less than the European call option because the asset price must cross the barrier for the option to become active.

### Lookback Options
Lookback options do not use a strike price but use the minimum or maximum asset price as their strike.  The discounted payoffs are

$$
\begin{array}{rcc}
& \textbf{call} & \textbf{put} \\ \hline
 \textbf{payoff} & 
 \displaystyle \Bigl(S(T) - \min_{0 \le t \le T} S(t),0 \Bigr)\mathsf{e}^{-rT} & 
 \displaystyle \Bigl(\max_{0 \le t \le T} S(t) - S(T),0 \Bigr)\mathsf{e}^{-rT} 
 \end{array}
$$

where the values of $t$ considered for the minimum or maximum are either discrete, $0, T/d, \dots, T$, or continuous.  Note that we would
expect the prices of these options to be greater than their out of the money European counterparts.

In [11]:
LookCall = qp.LookbackOption(qp.IIDStdUniform(dimension=d,seed=7), volatility= vol,start_price= initPrice,
                            interest_rate = interest,t_final=1/4,call_put='call')
sc = qp.CubMCCLT(LookCall,absTol)
solution, data = sc.integrate()
print("To reach the absolute error tolerance = ",absTol, ", we need to generate ", f"{int(data.n_total):,}",
      "iid points, the estimation of the fair price is",f"{solution:.4f}")

AttributeError: module 'qmcpy' has no attribute 'LookbackOption'

In [5]:
LookbackCall = qp.LookBackOption(qp.DigitalNetB2(4,seed=7))

TypeError: __init__() missing 1 required positional argument: 'dim_frac'